In [13]:
"""
Peatland Protection Interactive Map
Shows protected/unprotected peatlands in Northern Ireland with clickable popups.
Farms and trails are visible but not interactive.
"""
import geopandas as gpd  # for spatial data handling
import pandas as pd      # for dataframes and joining
import folium            # for interactive mapping
from shapely.geometry import LineString  # for distance line drawing


In [14]:
def load_and_prepare_data():
    """
    Load and reproject all spatial datasets to EPSG:4326.
    """
    peat = gpd.read_file("datafiles/Peatland/Priority_Habitats_-_Peatland.shp").to_crs(epsg=4326)  # peatland polygons
    assi = gpd.read_file("datafiles/ASSI/ASSI.shp").to_crs(epsg=4326)                              # protected ASSI areas
    aonb = gpd.read_file("datafiles/AONB/AONB.shp").to_crs(epsg=4326)                              # AONB areas
    trails = gpd.read_file("datafiles/Trails/offroadtrailsni.shp").to_crs(epsg=4326)              # trails network
    scheme_data = gpd.read_file("datafiles/Farm/SchemeData.shp").to_crs(epsg=4326)                 # farm scheme data

    return peat, assi, aonb, trails, scheme_data
    peat, assi, aonb, trails, scheme_data = load_and_prepare_data()

In [15]:
peat.crs 

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [16]:
assi.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [17]:
aonb.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [18]:
trails.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [19]:
scheme_data.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [1]:
def classify_peatland(peat, assi, aonb):
    """
    Intersects peat with ASSI and AONB to create protected layer.
    Remaining peat becomes unprotected layer.
    """
    peat_assi = gpd.overlay(peat, assi, how='intersection')    # peat inside ASSI
    peat_aonb = gpd.overlay(peat, aonb, how='intersection')    # peat inside AONB

    protected_peat = gpd.GeoDataFrame(pd.concat([peat_assi, peat_aonb], ignore_index=True), crs=peat.crs).copy()  # merge
    protected_peat["Area_ha"] = protected_peat.to_crs(epsg=3035).geometry.area / 10000  # calculate hectares (in EPSG:3035)

    unioned = protected_peat.geometry.union_all()  # create single protected area union
    unprotected_peat = peat[~peat.geometry.intersects(unioned)].copy()  # everything else
    unprotected_peat["Area_ha"] = unprotected_peat.to_crs(epsg=3035).geometry.area / 10000  # area in hectares

    return protected_peat, unprotected_peat
